In [1]:
import matplotlib
matplotlib.use('Agg')

%load_ext autoreload
%autoreload 2

%matplotlib tk
%autosave 180
import matplotlib.pyplot as plt
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

import numpy as np
import os
import umap
import parmap

import glob2
import numpy as np
from sklearn.decomposition import PCA

from shapely.geometry import Polygon
from shapely import geometry
from matplotlib.patches import Polygon
from scipy.spatial import ConvexHull
import pickle as pk


import sys
sys.path.append("/home/cat/code/widefieldPredict/") # Adds higher directory to python modules path.



# visualize results module
from visualize import visualize_svm as Visualize


# 
#data_dir = '/media/cat/4TBSSD/yuki/'
data_dir = '/media/cat/4TBSSD/yuki/'


Autosaving every 180 seconds


In [2]:
def generate_pca_umap_scatter_plots_sliding_window(start1, width, n_steps, data, data_r, umap_plot, fname):

    for k in range(n_steps):
        start = start1+k*width
        end = start+width
        data_all = data[:,:,start:end]
        data_all_r = data_r[:,:,start:end]

        if False:
            data_all = data_all.mean(1)
            data_all_r = data_all_r.mean(1)

        # average all singel trials
        mean = data_all.mean(0)

        print ("data_all: ", data_all.shape)
        print ("Mean: ", mean.shape)

        data_pca = data_all.reshape(data_all.shape[0],-1)
        print ("data_pca", data_pca.shape)

        data_pca_r = data_all_r.reshape(data_all.shape[0],-1)


        X = np.vstack((data_pca, data_pca_r))
        print ("data input: ", X.shape)


        #############################################
        #############################################
        #############################################
        #
        #if k==0:
        pca = PCA(n_components=2)
        pca.fit(X)


        p = pca.transform(X)
        print ("Pca output: ", p.shape)

        if umap_plot:
            import umap
            fit = umap.UMAP()
            u = fit.fit_transform(X)
            print ("Umap output: ", u.shape)


        # #######################################
        # ########### PLOT RESULTS ##############
        # #######################################

        dot_size = 100
        fig=plt.figure(figsize=(5,5))
        plt.scatter(p[:106,0],
                    p[:106,1],
                    s=dot_size,
                   c='blue',
                   alpha=.9,
                   label='lever')

        plt.scatter(p[106:,0],
                    p[106:,1],
                    c='black',
                    s=dot_size,
                    alpha=.9,
                    label='random')

        plt.legend()

        if False:
            plt.savefig('/home/cat/pca_fullvector_'+str(k-n_steps)+'.svg',dpi=1200)
            plt.close()
        else:
            plt.show()

        #ctr+=1

        if umap_plot:
            fig=plt.figure(figsize=(5,5))
            plt.scatter(u[:106,0],
                        u[:106,1],
                        s=dot_size,
                        c='blue',
                       alpha=.9)

            plt.scatter(u[106:,0],
                        u[106:,1],
                        s=dot_size,
                       c='black',
                       alpha=.9)

            plt.savefig('/home/cat/umap_fullvector_'+str(k-n_steps)+'.svg',dpi=1200)
            plt.close()
            
            
            
# 
def generate_var_explained_files_from_pkl(main_dir, animal_ids, session_id):
    #
    for animal_id in animal_ids: 
        sessions = get_sessions(main_dir,
                                 animal_id,
                                 session_id)

        # 
        for session in sessions:
            print ("session: ", session)

            # 
            fname_pca = os.path.join(main_dir,animal_id,'tif_files',session,
                                                 session+ '_whole_stack_trial_ROItimeCourses_15sec_pca.pkl')
            # 
            fname_out = fname_pca.replace('.pkl','_var_explained.npy')

            if os.path.exists(fname_out)==False:
                try:


                    file = open(fname_pca, 'rb')
                    pca = pk.load(file)

                    np.save(fname_out, pca.explained_variance_ratio_)
                except: 
                    print (" missing pkl file")
            print ('')

In [2]:
###############################################################
################# VISUALIZE PCA 2D MAPS #######################
###############################################################

# ***************** Fig 2A *********************

#    
mask = np.int32(np.loadtxt('/media/cat/4TBSSD/yuki/IJ2/genericmask.txt'))
contours = np.load('/media/cat/4TBSSD/yuki/meta_allenmap/contours_fig1.npy')
print (mask.shape)

mask2 = np.ones((128,128),'float32')
for k in range(mask.shape[0]):
    #print (mask[k])
    mask2[mask[k][0],mask[k][1]]=np.nan

# 
fig=plt.figure()
data = np.load('/media/cat/4TBSSD/yuki/IJ2/tif_files/IJ2pm_Feb5_30Hz/IJ2pm_Feb5_30Hz_code_04_trial_ROItimeCourses_30sec_pca_0.95_spatial.npy')
#print (data.shape)
data = data.reshape(data.shape[0],128,128)
for k in range(4):
    ax=plt.subplot(2,2,k+1)
    vmin = np.min(data[k])
    vmax = np.max(data[k])
    
    # 
    #print (k, vmin,vmax)
    plt.imshow(data[k]*mask2*contours, vmin=vmin, vmax=vmax)
    plt.xticks([])
    plt.yticks([])
    
if False:
    plt.savefig('/home/cat/pca_4comps.svg')
    plt.close()
else:
    plt.show()

(3917, 2)


In [9]:
########################################################
#### COMPUTE PCA AND UMAP FULL VECTOR SCATER PLOT ######
########################################################

# ******************** FIG 2D ***********************

data = np.load('/home/cat/concatenated_trials.npy')

fname = '/media/cat/4TBSSD/yuki/IJ2/tif_files/IJ2pm_Mar2_30Hz/IJ2pm_Mar2_30Hz_code_04_trial_ROItimeCourses_30sec_pca_0.95.npy'

data = np.load(fname)

data_r = np.load(fname.replace('trial','random'))
#data = np.load('/media/cat/4TBSSD/yuki/IJ2/tif_files/IJ2pm_Mar3_30Hz/IJ2pm_Mar3_30Hz_code_04_trial_ROItimeCourses_30sec_pca_0.95.npy')
#data = np.load('/media/cat/4TBSSD/yuki/IJ1/tif_files/IJ1pm_Mar2_30Hz/IJ1pm_Mar2_30Hz_code_04_trial_ROItimeCourses_30sec_pca_0.95.npy')
print ("raw data: ", data.shape)
print ("random data: ", data_r.shape)

#######################
umap_plot = False

# sliding window parameters can adjust windows and/or time steps
start1 = 750
width = 150
n_steps = 1

# Fig 2 scatter plots
generate_pca_umap_scatter_plots_sliding_window(start1, width, n_steps, data, data_r, umap_plot, fname)





raw data:  (106, 6, 1801)
random data:  (106, 6, 1801)
data_all:  (106, 6, 150)
Mean:  (6, 150)
data_pca (106, 900)
data input:  (212, 900)
Pca output:  (212, 2)


In [69]:
####################################################
####### GRAB VARIANCE EXPLAINED FOR ALL DATA #######
####################################################
# 
main_dir = '/media/cat/4TBSSD/yuki/'
animal_ids = ["IA1", "IA2", "IA3", "IJ1", "IJ2", "AQ2"]
session_id = 'all'

# 
generate_var_explained_files_from_pkl(main_dir, animal_ids, session_id)


############################################
animal_ids = ['IA1']
for animal_id in animal_ids: 
    sessions = get_sessions(main_dir,
                             animal_id,
                             session_id)
    # 
    for session in sessions:
        print ("session: ", session)

        # 
        fname_var = os.path.join(main_dir,animal_id,'tif_files',session,
                                             session+ '_whole_stack_trial_ROItimeCourses_15sec_pca_var_explained.npy')

        if os.path.exists(fname_var)==True:
        
        var = np.load(fname_var)
        
        plt.plot(var)
        
#      
plt.plot(pca.explained_variance_ratio_[:10])
plt.show()
        
        

session:  IA1pm_Feb1_30Hz

session:  IA1pm_Feb2_30Hz

session:  IA1pm_Feb3_30Hz

session:  IA1pm_Feb4_30Hz

session:  IA1pm_Feb5_30Hz

session:  IA1pm_Feb9_30Hz

session:  IA1pm_Feb10_30Hz

session:  IA1pm_Feb11_30Hz

session:  IA1pm_Feb12_30Hz

session:  IA1pm_Feb15_30Hz

session:  IA1pm_Feb16_30Hz

session:  IA1pm_Feb17_30Hz

session:  IA1pm_Feb18_30Hz

session:  IA1pm_Feb19_30Hz

session:  IA1pm_Feb22_30Hz

session:  IA1pm_Feb23_30Hz

session:  IA1pm_Feb24_30Hz

session:  IA1pm_Feb25_30Hz
 missing pkl file

session:  IA1pm_Feb26_30Hz

session:  IA1pm_Feb29_30Hz

session:  IA1pm_Mar1_30Hz

session:  IA1pm_Mar2_30Hz

session:  IA1pm_Mar3_30Hz

session:  IA1am_Mar4_30Hz

session:  IA1am_Mar7_30Hz

session:  IA1pm_Mar8_30Hz

session:  IA1am_Mar9_30Hz

session:  IA1am_Mar10_30Hz

session:  IA1am_Mar11_30Hz

session:  IA1pm_Mar14_30Hz

session:  IA1am_Mar15_30Hz

session:  IA1pm_Mar16_30Hz

session:  IA1pm_Mar17_30Hz

session:  IA1pm_Mar18_30Hz

session:  IA1pm_Mar21_30Hz

session:  IA1pm


session:  AQ2am_Jan18_30Hz

session:  AQ2am_Jan19_30Hz

session:  AQ2pm_Jan19_30Hz

session:  AQ2pm_Jan20_30Hz

session:  AQ2am_Jan20_30Hz

session:  AQ2pm_Jan21_30Hz

session:  AQ2am_Jan21_30Hz

session:  AQ2pm_Jan22_30Hz

session:  AQ2am_Jan22_30Hz

session:  AQ2am_Jan25_30Hz

session:  AQ2pm_Jan25_30Hz

session:  AQ2pm_Jan26_30Hz

session:  AQ2am_Jan26_30Hz

session:  AQ2am_Jan27_30Hz

session:  AQ2pm_Jan27_30Hz

session:  AQ2am_Jan28_30Hz

session:  AQ2pm_Jan28_30Hz

session:  AQ2am_Jan29_30Hz

session:  AQ2pm_Jan29_30Hz

session:  AQ2am_Feb2_30Hz

session:  AQ2am_Feb3_30Hz

session:  AQ2am_Feb4_30Hz

session:  AQ2am_Feb5_30Hz

session:  AQ2am_Feb9_30Hz

session:  AQ2am_Feb10_30Hz

session:  AQ2am_Feb11_30Hz

session:  AQ2am_Feb12_30Hz

session:  AQ2am_Feb15_30Hz

session:  AQ2am_Feb16_30Hz

session:  AQ2am_Feb17_30Hz

session:  AQ2am_Feb18_30Hz

session:  AQ2am_Feb19_30Hz

session:  AQ2am_Feb22_30Hz

session:  AQ2am_Feb23_30Hz

session:  AQ2am_Feb25_30Hz
 missing pkl file

sessio

In [ ]:
###################################################
### GENERATE ALL DATA VS. PULLS IN PCA SPACE ######
###################################################

# ************** FIG 2E Convex hull analysis
# from Visualize import (get_data_and_triggers, 
#                        get_pca_object_and_all_points,
#                        project_data_pca,
#                        plot_pca_scatter,
#                        get_convex_hull,
#                        plot_convex_hull,
#                        get_sessions)


root_dir = '/media/cat/4TBSSD/yuki/'

animal_ids = ['IA1','IA2','IA3','IJ1',"IJ2","AQ2"]
session_id = 'all'


# 
for animal_id in animal_ids:
    pa = Visualize.PCA_Analysis()
    pa.root_dir = '/media/cat/4TBSSD/yuki/'
    pa.session_id = 'all'
    pa.animal_id = animal_id

    # 
    pa.n_pca = 10
    pa.sliding_window = 30    # how many frames to take into analysis window
    pa.n_frames = 300         # how many frames back in time to analyze: 

    
    # 
    sessions = Visualize.get_sessions(pa.root_dir,
                            pa.animal_id,
                            pa.session_id)


    for session in sessions:
        pa.session = session
        print (" processing: ", session)
        fname_out = os.path.join(pa.root_dir,
                                  pa.animal_id,
                                  'tif_files',
                                  pa.session,
                                  pa.session+'_convex_hull.npz')
        if os.path.exists(fname_out):
            print ("     data laready processed")
            print ('')
            continue
            
        fname_triggers = os.path.join(pa.root_dir,
                                      pa.animal_id,
                                      'tif_files',
                                      pa.session,
                                      'blue_light_frame_triggers.npz')

        fname_data = os.path.join(pa.root_dir,
                                     pa.animal_id,
                                     'tif_files',
                                     pa.session,
                                     pa.session+ '_aligned_butterworth_0.1hz_6.0hz.npy')

        pa.X, pa.triggers = Visualize.get_data_and_triggers(pa)
        
        if pa.X is None:
            print ("     missing data/blue light")
            print ('')
            print ('')
            continue
            
        if pa.triggers.shape[0]<5:
            print ("     not enough points : ", pa.triggers.shape )
            print ('')
            print ('')
            continue
            
        # make PCA plots
        pa.pca, pa.all_points = Visualize.get_pca_object_and_all_points(pa)


        # FIG 2 E top
        pa.p_lever =  Visualize.project_data_pca(pa)

        # skip plotting option
        # plot_pca_scatter(pa)
         
        # FIG 2E
        pa =  Visualize.get_convex_hull(pa)

        # skip plotting option
        # plot_convex_hull(pa)
        print ('')


 processing:  IA1pm_Feb1_30Hz
     not enough points :  (3,)


 processing:  IA1pm_Feb2_30Hz
     data laready processed

 processing:  IA1pm_Feb3_30Hz
     data laready processed

 processing:  IA1pm_Feb4_30Hz
 data missing/led missing
     missing data/blue light


 processing:  IA1pm_Feb5_30Hz
     not enough points :  (4,)


 processing:  IA1pm_Feb9_30Hz
     data laready processed

 processing:  IA1pm_Feb10_30Hz
     data laready processed

 processing:  IA1pm_Feb11_30Hz
     data laready processed

 processing:  IA1pm_Feb12_30Hz
 data missing/led missing
     missing data/blue light


 processing:  IA1pm_Feb15_30Hz
     data laready processed

 processing:  IA1pm_Feb16_30Hz
     data laready processed

 processing:  IA1pm_Feb17_30Hz
     data laready processed

 processing:  IA1pm_Feb18_30Hz
     data laready processed

 processing:  IA1pm_Feb19_30Hz
     data laready processed

 processing:  IA1pm_Feb22_30Hz


In [72]:
#############################
##### PLOT CONVEX HULL ######
#############################

# Fig 2 E

fname = '/media/cat/4TBSSD/yuki/IA1/tif_files/IA1pm_Feb3_30Hz/IA1pm_Feb3_30Hz_convex_hull.npz'
d = np.load(fname, allow_pickle = True)

animal_id = 'IJ1'

pa = PCA_Analysis()
pa.root_dir = '/media/cat/4TBSSD/yuki/'
pa.session_id = 'Feb3_'

pa.n_pca = 10
pa.sliding_window = 30    # how many frames to take into analysis window
pa.n_frames = 300         # how many frames back in time to analyze: 

pa.animal_id = animal_id
pa.ratio_single = d['ratio_single'][::-1]
pa.ratio_cumsum = d['ratio_cumsum'][::-1]
pa.ratio_random_single = d['ratio_random_single'][::-1]
pa.ratio_random_cumulative = d['ratio_random_cumulative'][::-1]


pa.session = get_sessions(pa.root_dir,
                      pa.animal_id,
                      pa.session_id)[0]

print (" processing: ", pa.session)
fname_out = os.path.join(pa.root_dir,
                          pa.animal_id,
                          'tif_files',
                          pa.session,
                          pa.session+'_convex_hull.npz')



pa.X, pa.triggers = get_data_and_triggers(pa)

if pa.X is None:
    print ("     missing data/blue light")
    print ('')
    print ('')

elif pa.triggers.shape[0]<5:
    print ("     not enough points : ", pa.triggers.shape )
    print ('')
    print ('')
else:
    # make PCA plots
    pa.pca, pa.all_points = get_pca_object_and_all_points(pa)


    # FIG 2 E top
    pa.p_lever =  project_data_pca(pa)


    # scatter plot option
    plot_pca_scatter(pa)

        
# plot_convex_hull(pa)

 processing:  IJ1pm_Feb3_30Hz


  0%|          | 0/300 [00:00<?, ?it/s]

 X data using :  30  number of frames  (1301, 491520)
300 31 10


100%|██████████| 300/300 [00:56<00:00,  5.34it/s]


(1301,)
plever:  (289, 31, 10)


In [74]:
##########################################################
#### GENERATE PCA DISTRIBUTIONS OF ALL BODY MOVEMENTS ####
##########################################################

#  Fig 3E 
pa = PCA_Analysis()
pa.root_dir = '/media/cat/4TBSSD/yuki/'
pa.n_pca = 10
pa.sliding_window = 30    # how many frames to take into analysis window
pa.n_frames = 1         # how many frames back in time to analyze: 

# 
pa.umap_flag = False
pa.convex_hull_flag = False

pa.frame_rate = 30

########################################
animal_ids = ['IA1','IA2','IA3','IJ1',"IJ2","AQ2"]
#animal_ids = ['IA2']
for animal_id in animal_ids:
    
    fname_out = os.path.join(pa.root_dir, animal_id,'tif_files',
                            'res_simplex.npy')

    if os.path.exists(fname_out)==False:
        pa.animal_id = animal_id
        session = 'all'

        pa.plot_all = True
        pa.plot_3D = False
        pa.plot_flag = False


        #########################
        # get the correct sesion name:
        sessions = get_sessions(pa.root_dir,
                                  pa.animal_id,
                                  session)

        #print ("sessions: ", sessions)
        # 
        res_simplex = pca_scatter_body_movements_fig3(pa, sessions)

        np.save(fname_out, res_simplex)
    
    
print (" COMPLETED: SESSIONS: ", )

 COMPLETED: SESSIONS: 


In [85]:
##################################################
### PLOT CONVEX HULLS FOR BODY PARTS VS LEVER ####
##################################################

# # Fig 3E
 
lefts = []
rights = []
levers = []
nTrials = []
animal_ids = ['IA1','IA2','IA3','IJ1',"IJ2","AQ2"]

animal_id = ['IJ2']
session = 'Mar2_'
#session = 'all'
for animal_id in animal_ids:

    #fig=plt.figure()

    # 
    fname_simplex = os.path.join(pa.root_dir, animal_id,'tif_files/res_simplex.npy')
    res_simplex = np.load(fname_simplex,allow_pickle=True)
    
    #print ("ressimplex ", res_simplex)
    
    # get the correct sesion name:
    sessions = get_sessions(pa.root_dir,
                            animal_id,
                            session)
    print ("sessions: ", sessions)

    (lever_vs_left_paw, 
     lever_vs_right_paw, 
     lever_vs_all, 
     n_trials) = plot_intersection_convex_hulls_lever_vs_bodyparts(res_simplex,
                                                                   sessions,
                                                                   animal_id,
                                                                   pa.root_dir)
    plt.suptitle(animal_id)

#     if False:
#         plt.savefig('/home/cat/polygons.png',dpi=300)
#         plt.close()
#     else:
#         plt.show()

    # 
    lefts.append(np.array(lever_vs_left_paw))
    rights.append(np.array(lever_vs_right_paw))
    levers.append(np.array(lever_vs_all))
    nTrials.append(np.array(n_trials))
#     # 
#     fig=plt.figure()
#     idx = np.where(np.isnan(left)==False)[0]
#     t = np.arange(len(left))[idx]
#     plt.scatter(t, left[idx])


#     # 
#     idx = np.where(np.isnan(right)==False)[0]
#     t = np.arange(len(right))[idx]
#     plt.scatter(t, right[idx])
#     plt.suptitle(animal_id)
plt.show()

sessions:  ['IA1pm_Mar2_30Hz']
all areas:  [1544226526.9314482, 1601138774.5727801, 1572212687.1700273, 1487279289.7549741, 1628631202.044745, nan]








IndexError: index 1 is out of bounds for axis 0 with size 1

In [82]:
##########################################
######## OVERLAP POLYGON RATIOS ##########
##########################################

# Fig 3J

from sklearn.linear_model import LinearRegression


def find_p_t(model, X, y, X2, y_pred):
    #self = super(LinearRegression, self).fit(X, y, n_jobs)
    #print (self)

    #print (y_pred.shape, y.shape)
    sse = np.sum((y_pred - y) ** 2, axis=0) / float(X.shape[0] - X.shape[1])
    
    #print (sse)
    se = np.array([
        np.sqrt(np.diagonal(sse[i] * np.linalg.inv(np.dot(X.T, X))))
                                                for i in range(sse.shape[0])
                ])

    t = model.coef_ / se
    p = 2 * (1 - stats.t.cdf(np.abs(t), y.shape[0] - X.shape[1]))
    
    return t, p


clrs=['black','blue','red','green','magenta','pink']
sig = [0.05,0.01,0.001, 0.0001, 0.00001]
animal_ids = ['M1','M2','M3','M4','M5','M6']

fig=plt.figure()
for k in range(len(lefts)):
    temp = levers[k]
    
    # normalize to n trials per session
    if True:
        temp = temp/nTrials[k]
        
    print (temp.shape)
    
        
    idx = np.where(np.isnan(temp)==False)[0]
    #t=t[idx]
    temp=temp[idx]
    
    
    idx = np.where(temp!=0)[0]
    #t=t[idx]
    temp=temp[idx]
    
    
    model = LinearRegression()
    y=np.array(temp).reshape(-1, 1)
    x = np.arange(y.shape[0]).reshape(-1, 1)
    model.fit(x, y)

    x2 = np.arange(0,y.shape[0],1).reshape(-1, 1)
    y_pred = model.intercept_ + model.coef_ * x2


    t,p = find_p_t(model, x, y, x2, y_pred)
    
    temp_sig = ''
    for j in range(len(sig)):
        if p <= sig[j]:
            temp_sig = temp_sig+"*"
            
    
    plt.plot(x2, y_pred, label=animal_ids[k]+ " "+temp_sig,
             c=clrs[k],
            linewidth=6)
    
    
    
    t = np.arange(temp.shape[0])
    plt.scatter(t,temp,
                s=100,
                edgecolor='black',
               c=clrs[k],
               alpha=.7)

    
plt.legend(fontsize=12)
plt.xlim(0,43)
plt.ylim(bottom=0)
            
plt.show()

(24,)
(10,)
(11,)
(10,)
(9,)
(44,)


In [139]:
import sphviewer as sph
from numpy.ma import masked_array

def myplot(x, y, nb=32, xsize=500, ysize=500):   
    xmin = np.min(x)
    xmax = np.max(x)
    ymin = np.min(y)
    ymax = np.max(y)

    x0 = (xmin+xmax)/2.
    y0 = (ymin+ymax)/2.

    pos = np.zeros([len(x),3])
    pos[:,0] = x
    pos[:,1] = y
    w = np.ones(len(x))

    P = sph.Particles(pos, w, nb=nb)
    S = sph.Scene(P)
    S.update_camera(r='infinity', x=x0, y=y0, z=0, 
                    xsize=xsize, ysize=ysize)
    R = sph.Render(S)
    R.set_logscale()
    img = R.get_image()
    extent = R.get_extent()
    for i, j in zip(range(4), [x0,x0,y0,y0]):
        extent[i] += j
    print (extent)
    
    
    clrs = np.unique(img)
    img_out = np.zeros(img.shape,'float32')
    for k in range(clrs.shape[0]):
        idx = np.where(img==clrs[k])
        print (k)
        img_out[idx]=k
    
    return img_out, extent

# Generate Lever locations
print (pa.p_lever.shape)
x = pa.p_lever.squeeze()[:,0]
y = pa.p_lever.squeeze()[:,1]
x_size = 100
y_size = 100
heatmap_lever, extent_lever = myplot(x,y, nb=4, xsize=x_size, ysize=y_size)

print ("npm ax: ", np.max(heatmap_lever), np.min(heatmap_lever))
print (np.unique(heatmap_lever))

v1_lever = masked_array(heatmap_lever,
                   heatmap_lever>=1)

# Generate left paw locations
print (pa_array[0].shape)
x = pa_array[0].squeeze()[:,0]
y = pa_array[0].squeeze()[:,1]
x_size = 100
y_size = 100
heatmap_leftpaw, extent_left_paw = myplot(x,y, nb=4, xsize=x_size, ysize=y_size)

print ("npm ax: ", np.max(heatmap_leftpaw), np.min(heatmap_leftpaw))
print (np.unique(heatmap_leftpaw))



fig,ax = plt.subplots()

#ax4.imshow(heatmap_64, 
ax1=plt.subplot(121)  
p1 = ax1.imshow(heatmap_lever,
               extent=extent_lever, 
               #origin='lower', 
               aspect='auto',
               cmap='Reds')

plt.xlim(-40000,60000)
plt.ylim(-50000,50000)

#cba = plt.colorbar(pa,shrink=0.25)

# 
ax2=plt.subplot(122)  
# v1_leftpaw = masked_array(heatmap_leftpaw,
#                           heatmap_leftpaw>=1)
p2 = ax2.imshow(heatmap_leftpaw, 
               extent=extent_left_paw, 
               #origin='lower', 
               aspect='auto',
               cmap='Greys')

plt.xlim(-40000,60000)
plt.ylim(-50000,50000)


#cbb = plt.colorbar(pb,shrink=0.25)
#plt.xlabel('Day')
#plt.ylabel('Depth')
#cba.set_label('positive')
#cbb.set_label('negative')
#plt.show()



plt.show()



(1, 31, 10)
[-41973.54    8432.656 -18250.133  34552.848]
0
1
2
npm ax:  2.0 0.0
[0. 1. 2.]
(1, 55, 10)
[-34157.016 265766.25  -61826.69   18024.137]
0
1
2
3
4
npm ax:  4.0 0.0
[0. 1. 2. 3. 4.]


In [90]:
all_points = np.load('/media/cat/4TBSSD/yuki/IJ2/tif_files/IJ2pm_Feb1_30Hz/IJ2pm_Feb1_30Hz_all_points.npy')
print (all_points.shape)

plt.scatter(all_points[:,0],
           all_points[:,1])
plt.show()

(1297, 10)
